# This notebook is for course Five : Regression (Supervised Learning)

### Importing Data Files

In [ ]:
#Import Product DataSet here
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3
product_data = pd.read_csv(body,sep='|')
product_data.head()

In [1]:
# The code was removed by Watson Studio for sharing.

,PRODUCT CODE,PRODUCT CATEGORY,UNIT LIST PRICE
0,30001,HEALTH & BEAUTY,$7.45
1,30002,HEALTH & BEAUTY,$5.35
2,30003,HEALTH & BEAUTY,$5.49
3,30004,HEALTH & BEAUTY,$6.46
4,30005,HEALTH & BEAUTY,$7.33


In [2]:
#Import Transaction DataSet Here
body = client_4c73b4e9125d48cf93a58e32e4df6b8a.get_object(Bucket='datascienceproject-donotdelete-pr-knsi3l5zevhael',Key='Transaction Data Set - Student 3 of 3.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

transaction_data = pd.read_csv(body,sep='|')
transaction_data.head()

,CUSTOMER NUM,PRODUCT NUM,QUANTITY PURCHASED,DISCOUNT TAKEN,TRANSACTION DATE,STOCKOUT
0,10114,30011,4,0.0,1/2/2015,0
1,10217,30016,3,0.0,1/2/2015,0
2,10224,30013,4,0.0,1/2/2015,0
3,10103,30012,3,0.2,1/2/2015,0
4,10037,30010,8,0.0,1/2/2015,0


In [3]:
#Import Customer Dataset Here
body = client_4c73b4e9125d48cf93a58e32e4df6b8a.get_object(Bucket='datascienceproject-donotdelete-pr-knsi3l5zevhael',Key='Customer Data Set - Student 1 of 3.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

customer_data=pd.read_csv(body)
customer_data.head()


,CUSTOMERID,GENDER,AGE,INCOME,EXPERIENCE SCORE,LOYALTY GROUP,ENROLLMENT DATE,HOUSEHOLD SIZE,MARITAL STATUS
0,10001,0,64,"$133,498",5,enrolled,06-03-2013,4,Single
1,10002,0,42,"$94,475",9,notenrolled,NaN,6,Married
2,10003,0,40,"$88,610",9,enrolled,02-09-2010,5,Married
3,10004,0,38,"$84,313",8,enrolled,06-04-2015,1,Single
4,10005,0,30,"$51,498",3,notenrolled,NaN,1,Single


### Changing Data Types

In [4]:
customer_data['INCOME']=customer_data['INCOME'].map(lambda x : x.replace('$',''))

In [5]:
customer_data['INCOME']=customer_data['INCOME'].map(lambda x : int(x.replace(',','')))

### Creating Customer View

In [13]:
trans_products=transaction_data.merge(product_data,how='inner', left_on='PRODUCT NUM', right_on='PRODUCT CODE')

In [14]:
trans_products.head()

,CUSTOMER NUM,PRODUCT NUM,QUANTITY PURCHASED,DISCOUNT TAKEN,TRANSACTION DATE,STOCKOUT,PRODUCT CODE,PRODUCT CATEGORY,UNIT LIST PRICE
0,10114,30011,4,0.0,1/2/2015,0,30011,APPAREL,$25.46
1,10086,30011,6,0.0,1/2/2015,0,30011,APPAREL,$25.46
2,10174,30011,10,0.0,1/2/2015,0,30011,APPAREL,$25.46
3,10401,30011,12,0.0,1/2/2015,0,30011,APPAREL,$25.46
4,10216,30011,12,0.1,1/2/2015,0,30011,APPAREL,$25.46


In [15]:
trans_products['UNIT LIST PRICE']=trans_products['UNIT LIST PRICE'].map(lambda x : float(x.replace('$','')))

In [16]:
trans_products.head()

,CUSTOMER NUM,PRODUCT NUM,QUANTITY PURCHASED,DISCOUNT TAKEN,TRANSACTION DATE,STOCKOUT,PRODUCT CODE,PRODUCT CATEGORY,UNIT LIST PRICE
0,10114,30011,4,0.0,1/2/2015,0,30011,APPAREL,25.46
1,10086,30011,6,0.0,1/2/2015,0,30011,APPAREL,25.46
2,10174,30011,10,0.0,1/2/2015,0,30011,APPAREL,25.46
3,10401,30011,12,0.0,1/2/2015,0,30011,APPAREL,25.46
4,10216,30011,12,0.1,1/2/2015,0,30011,APPAREL,25.46


In [17]:
trans_products['Total_Price']=trans_products['QUANTITY PURCHASED'] * trans_products['UNIT LIST PRICE'] * (1- trans_products['DISCOUNT TAKEN'])

In [18]:
trans_products.head()

,CUSTOMER NUM,PRODUCT NUM,QUANTITY PURCHASED,DISCOUNT TAKEN,TRANSACTION DATE,STOCKOUT,PRODUCT CODE,PRODUCT CATEGORY,UNIT LIST PRICE,Total_Price
0,10114,30011,4,0.0,1/2/2015,0,30011,APPAREL,25.46,101.840
1,10086,30011,6,0.0,1/2/2015,0,30011,APPAREL,25.46,152.760
2,10174,30011,10,0.0,1/2/2015,0,30011,APPAREL,25.46,254.600
3,10401,30011,12,0.0,1/2/2015,0,30011,APPAREL,25.46,305.520
4,10216,30011,12,0.1,1/2/2015,0,30011,APPAREL,25.46,274.968


In [19]:
customer_prod_categ=trans_products.groupby(['CUSTOMER NUM','PRODUCT CATEGORY']).agg({'Total_Price':'sum'})

In [20]:
customer_prod_categ.head()

Total_Price
CUSTOMER NUM PRODUCT CATEGORY             
10001        APPAREL              4022.430
             ELECTRONICS          1601.315
             FOOD                   68.688
             HEALTH & BEAUTY      1134.337
10002        APPAREL              2312.509

In [21]:
customer_prod_categ=customer_prod_categ.reset_index()

In [23]:
customer_prod_categ.head()

,CUSTOMER NUM,PRODUCT CATEGORY,Total_Price
0,10001,APPAREL,4022.430
1,10001,ELECTRONICS,1601.315
2,10001,FOOD,68.688
3,10001,HEALTH & BEAUTY,1134.337
4,10002,APPAREL,2312.509


In [24]:
customer_pivot=customer_prod_categ.pivot(index='CUSTOMER NUM',columns='PRODUCT CATEGORY',values='Total_Price')

In [25]:
customer_pivot.head()

PRODUCT CATEGORY,APPAREL,ELECTRONICS,FOOD,HEALTH & BEAUTY
CUSTOMER NUM,,,,
10001,4022.430,1601.315,68.688,1134.337
10002,2312.509,2473.163,276.779,NaN
10003,2887.382,5414.418,260.640,NaN
10004,3637.213,1840.211,45.270,NaN
10005,213.512,NaN,NaN,NaN


In [26]:
trans_total_spend=trans_products.groupby('CUSTOMER NUM').agg({'Total_Price':'sum'}).\
rename(columns={'Total_Price':'TOTAL SPENT'})

In [27]:
trans_total_spend.head()

,TOTAL SPENT
CUSTOMER NUM,
10001,6826.770
10002,5062.451
10003,8562.440
10004,5522.694
10005,213.512


In [28]:
customer_KPIs=customer_pivot.merge(trans_total_spend,how='inner',left_index=True, right_index=True )

In [29]:
customer_KPIs.head()

,APPAREL,ELECTRONICS,FOOD,HEALTH & BEAUTY,TOTAL SPENT
CUSTOMER NUM,,,,,
10001,4022.430,1601.315,68.688,1134.337,6826.770
10002,2312.509,2473.163,276.779,NaN,5062.451
10003,2887.382,5414.418,260.640,NaN,8562.440
10004,3637.213,1840.211,45.270,NaN,5522.694
10005,213.512,NaN,NaN,NaN,213.512


In [30]:
customer_KPIs=customer_KPIs.fillna(0)


In [31]:
customer_KPIs.head()

,APPAREL,ELECTRONICS,FOOD,HEALTH & BEAUTY,TOTAL SPENT
CUSTOMER NUM,,,,,
10001,4022.430,1601.315,68.688,1134.337,6826.770
10002,2312.509,2473.163,276.779,0.000,5062.451
10003,2887.382,5414.418,260.640,0.000,8562.440
10004,3637.213,1840.211,45.270,0.000,5522.694
10005,213.512,0.000,0.000,0.000,213.512


In [32]:
customer_all_view=customer_data.merge(customer_KPIs,how='inner', left_on='CUSTOMERID', right_index=True)

In [33]:
customer_all_view.head()

,CUSTOMERID,GENDER,AGE,INCOME,EXPERIENCE SCORE,LOYALTY GROUP,ENROLLMENT DATE,HOUSEHOLD SIZE,MARITAL STATUS,APPAREL,ELECTRONICS,FOOD,HEALTH & BEAUTY,TOTAL SPENT
0,10001,0,64,133498,5,enrolled,06-03-2013,4,Single,4022.430,1601.315,68.688,1134.337,6826.770
1,10002,0,42,94475,9,notenrolled,NaN,6,Married,2312.509,2473.163,276.779,0.000,5062.451
2,10003,0,40,88610,9,enrolled,02-09-2010,5,Married,2887.382,5414.418,260.640,0.000,8562.440
3,10004,0,38,84313,8,enrolled,06-04-2015,1,Single,3637.213,1840.211,45.270,0.000,5522.694
4,10005,0,30,51498,3,notenrolled,NaN,1,Single,213.512,0.000,0.000,0.000,213.512


# Linear Regression 

In [ ]:
#Begin Writing your code here

In [34]:
#select the following variables to the linear regression model to predict the total spending
#: gender, age, income, experience score and loyalty enrolment, household size, and marital status.
DF_input=customer_all_view[['GENDER','AGE','INCOME','EXPERIENCE SCORE','LOYALTY GROUP','HOUSEHOLD SIZE','MARITAL STATUS']]
DF_input.head()

,GENDER,AGE,INCOME,EXPERIENCE SCORE,LOYALTY GROUP,HOUSEHOLD SIZE,MARITAL STATUS
0,0,64,133498,5,enrolled,4,Single
1,0,42,94475,9,notenrolled,6,Married
2,0,40,88610,9,enrolled,5,Married
3,0,38,84313,8,enrolled,1,Single
4,0,30,51498,3,notenrolled,1,Single


In [35]:
#As you can see, Loyalty group and Marital status are strings, so they must be converted to numeric format 
#because the scikit-learn machine learning libraries accept features that are in numeric format. 
#So, we must encode both variables in a numeric format.
#The Marital status variable has four values: Single, Married, Divorced, and Widow/Widower. 
#Check the marital status by using the following command:

DF_input["MARITAL STATUS"].unique()


array(['Single', 'Married', 'Divorced', 'Widow/Widower'], dtype=object)

In [38]:
#Pandas has a method that is called “get_dummies” that detects categorical variables in a 
#Pandas data frame and automatically performs one hot encoding of it(creating a column and assigning 
#a '1' when the column title is true ). To get an idea about 
#Pandas one hot encoding, run the following command.
pd.get_dummies(DF_input["MARITAL STATUS"]).head(10)
#this coding will be incorporated ito the pandas data frame to highlight the status of each customer

,Divorced,Married,Single,Widow/Widower
0,0,0,1,0
1,0,1,0,0
2,0,1,0,0
3,0,0,1,0
4,0,0,1,0
5,0,1,0,0
6,0,1,0,0
7,0,0,1,0
8,0,1,0,0
9,0,1,0,0


In [40]:
#The Loyalty group has two values: “enrolled” and “notenrolled”. The encoding of this variable 
#is easy(binary encoding) and can be accomplished by using a simple function instead of the Pandas “pd.get_dummies“ function. 
#Run this code snippet function in a new cell to binary encode the loyalty variable:

def encode_loyalty(value):
    if value == "enrolled" :
        return 1 
    else : 
        return 0
DF_input['LOYALTY GROUP']=DF_input['LOYALTY GROUP'].apply(encode_loyalty)
DF_input.head()
#ignore the exception

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,GENDER,AGE,INCOME,EXPERIENCE SCORE,LOYALTY GROUP,HOUSEHOLD SIZE,MARITAL STATUS
0,0,64,133498,5,1,4,Single
1,0,42,94475,9,0,6,Married
2,0,40,88610,9,1,5,Married
3,0,38,84313,8,1,1,Single
4,0,30,51498,3,0,1,Single


In [41]:
#replacing the “DF_input” frame with the one-hot encoded Pandas data frame by running the following command:

DF_input=pd.get_dummies(DF_input)
DF_input.head()

,GENDER,AGE,INCOME,EXPERIENCE SCORE,LOYALTY GROUP,HOUSEHOLD SIZE,MARITAL STATUS_Divorced,MARITAL STATUS_Married,MARITAL STATUS_Single,MARITAL STATUS_Widow/Widower
0,0,64,133498,5,1,4,0,0,1,0
1,0,42,94475,9,0,6,0,1,0,0
2,0,40,88610,9,1,5,0,1,0,0
3,0,38,84313,8,1,1,0,0,1,0
4,0,30,51498,3,0,1,0,0,1,0


In [44]:
#Before feeding the data frame to the LinearRegession Scikit-learn library, scale the features to the 0 - 1 
#range because every input variable has its own scale. 
#To do this, Import the pre-processing library and save the Pandas columns to re-create the Pandas data frame 
#for the NumPy format by running the following command (the minmax scale command outputs NumPy):

from sklearn import preprocessing
DF_input_column_names = DF_input.columns.values
DF_input_np=preprocessing.minmax_scale(DF_input)
Reg_input_scaled=pd.DataFrame(DF_input_np, columns=DF_input_column_names)
Reg_input_scaled.head()


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:8: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64.


,GENDER,AGE,INCOME,EXPERIENCE SCORE,LOYALTY GROUP,HOUSEHOLD SIZE,MARITAL STATUS_Divorced,MARITAL STATUS_Married,MARITAL STATUS_Single,MARITAL STATUS_Widow/Widower
0,0.0,0.638889,0.872818,0.444444,1.0,0.6,0.0,0.0,1.0,0.0
1,0.0,0.333333,0.572046,0.888889,0.0,1.0,0.0,1.0,0.0,0.0
2,0.0,0.305556,0.526842,0.888889,1.0,0.8,0.0,1.0,0.0,0.0
3,0.0,0.277778,0.493722,0.777778,1.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.166667,0.240799,0.222222,0.0,0.0,0.0,0.0,1.0,0.0


In [45]:
#Create a train – test split of 80% training and 20% testing by running the following command:

from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(Reg_input_scaled,customer_all_view['TOTAL SPENT'], test_size=0.2,random_state=42)

In [46]:
#Import the LinearRegression Library by running the following command:

from sklearn.linear_model import LinearRegression

#Instantiate the model by running the following command:

regr = LinearRegression()

#Train the model on the training data set by running the following command::

regr.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [47]:
#Now that training is complete, you can predict the values of total spending for the testing feature 
#set by running the following command:

Y_pred = regr.predict(X_test)

In [49]:
#Obtain the regression coefficients of the linear model by running the “regr.coef_” method:

print('Coefficients: \n', regr.coef_)
#The importance of a predictor can be measured by the magnitude of its regression coefficient. 
#High-magnitude coefficients mean high importance and lower magnitudes means less importance regarding 
#target variables( (ignore sign because sign just means the direction of the correlation).
#High importance to low importance: The order is Income, Age, Loyalty Group, Household size,
#Experience score, Marital Status, and Gender.

Coefficients: 
 [   35.58494877 -7076.18001534 12403.0952701    505.84578936
  1566.48092344  1040.19923196   377.33573376   -82.02665324
   -34.57620088  -260.73287964]


In [50]:
#Obtain the constant or intercept term of the equation by running the following command:

print('intercept: \n', regr.intercept_)

intercept: 
 -38.16947521866132


In [51]:
#Thus the The linear regression equation to predict total spending is as follows:

#Predicted Total Spend = 35.58 * Gender - 7076 * AGE + 12403 * Income + 505 * Experience score + 155 * Loyalty group + 1040 * House hold size + 377.3 * (Marital Status – Divorced) – 82.02 * (Marital Status – Married) - 35.57 * (Marital Status – Single) - 260 * (Marital Status – Widow) - 38.17

In [52]:
#Evaluate the performance of the linear model by calculating the mean squared error. 
#Import its library and then apply the function to Predicted and actual values of customer 
#spending by running the following command:

from sklearn.metrics import mean_squared_error
print("Mean squared error: %.2f" % mean_squared_error(Y_test, Y_pred))
#the mean squared error is 3164098.64 which means the error is 1779. 
#Thus the model prdicts the total spending with an error of $1779.

Mean squared error: 3164098.64


In [53]:
#In the project, you used every customer feature as input to the linear regression model and 
#determined whether these features have a strong or weak relationship with customer spending. 
#Now, perform a bivariate analysis between each feature and the total spending variable and 
#Select only the relevant features (the features that have a strong relationship with customer spending) 
#and retrain the linear regression model by using only these features. Then, evaluate the model by using 
#the mean squared error metric.

In [54]:
DF_input=customer_all_view[['AGE','INCOME','LOYALTY GROUP','HOUSEHOLD SIZE']]
DF_input.head()


,AGE,INCOME,LOYALTY GROUP,HOUSEHOLD SIZE
0,64,133498,enrolled,4
1,42,94475,notenrolled,6
2,40,88610,enrolled,5
3,38,84313,enrolled,1
4,30,51498,notenrolled,1


In [55]:
def encode_loyalty(value):
    if value == "enrolled" :
        return 1 
    else : 
        return 0
DF_input['LOYALTY GROUP']=DF_input['LOYALTY GROUP'].apply(encode_loyalty)
DF_input.head()


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,AGE,INCOME,LOYALTY GROUP,HOUSEHOLD SIZE
0,64,133498,1,4
1,42,94475,0,6
2,40,88610,1,5
3,38,84313,1,1
4,30,51498,0,1


In [56]:
from sklearn import preprocessing
DF_input_column_names = DF_input.columns.values
DF_input_np=preprocessing.minmax_scale(DF_input)
Reg_input_scaled=pd.DataFrame(DF_input_np, columns=DF_input_column_names)
Reg_input_scaled.head()


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DataConversionWarning: Data with input dtype int64 were all converted to float64.
  app.launch_new_instance()


,AGE,INCOME,LOYALTY GROUP,HOUSEHOLD SIZE
0,0.638889,0.872818,1.0,0.6
1,0.333333,0.572046,0.0,1.0
2,0.305556,0.526842,1.0,0.8
3,0.277778,0.493722,1.0,0.0
4,0.166667,0.240799,0.0,0.0


In [57]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(Reg_input_scaled,customer_all_view['TOTAL SPENT'], test_size=0.2,random_state=42)


In [58]:
from sklearn.linear_model import LinearRegression
#Instantiate the model by running the following command:
regr = LinearRegression()
#Train the model on the training data set by running the following command::
regr.fit(X_train, Y_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [59]:
Y_pred = regr.predict(X_test)

In [60]:
print('Coefficients: \n', regr.coef_)
print('intercept: \n', regr.intercept_)


Coefficients: 
 [-7170.27726484 12496.40444648  1652.00843692   917.88476273]
intercept: 
 239.3234097961722


In [61]:
from sklearn.metrics import mean_squared_error
print("Mean squared error: %.2f" % mean_squared_error(Y_test, Y_pred))


Mean squared error: 3186316.30


In [ ]:
#Take away, the more the predictprs the better?